In [1]:
import time, datetime

In [2]:
datetime.datetime.now()

datetime.datetime(2018, 2, 7, 22, 22, 16, 487558)

In [3]:
MAX_FEATURES = 100000
SUB_FILE = 'submissions/sub_{}.csv'.format(int(time.time()))
SUB_FILE

'submissions/sub_1518042136.csv'

In [4]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
%matplotlib inline

In [6]:
train = pd.read_csv('../../toxic/data/train.csv')

In [7]:
test = pd.read_csv('../../toxic/data/test.csv')

In [8]:
subm = pd.read_csv('../../toxic/data/sample_submission.csv')

In [9]:
train.shape

(159571, 8)

In [10]:
test.shape

(153164, 2)

In [11]:
train.comment_text.fillna('unknown', inplace=True)

In [12]:
test.comment_text.fillna('unknown', inplace=True)

In [13]:
import string
special_punc = '“”¨«»®´·º½¾¿¡§£₤‘’'
all_punc = special_punc+string.punctuation

# replace punctuations

In [14]:
train.comment_text = train.comment_text.str.replace('[{}]'.format(all_punc),'')

In [15]:
test.comment_text = test.comment_text.str.replace('[{}]'.format(all_punc),'')

In [16]:
train.comment_text[0]

'Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted They werent vandalisms just closure on some GAs after I voted at New York Dolls FAC And please dont remove the template from the talk page since Im retired now892053827'

In [17]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', sublinear_tf=True, max_features=MAX_FEATURES)

In [18]:
train_term_doc = vec.fit_transform(train.comment_text)

In [19]:
test_term_doc = vec.transform(test.comment_text)

In [20]:
train_term_doc, test_term_doc

(<159571x100000 sparse matrix of type '<class 'numpy.float64'>'
 	with 12177756 stored elements in Compressed Sparse Row format>,
 <153164x100000 sparse matrix of type '<class 'numpy.float64'>'
 	with 9979860 stored elements in Compressed Sparse Row format>)

In [21]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [31]:
from sklearn.naive_bayes import MultinomialNB

In [32]:
clf = MultinomialNB()

In [33]:
clf.fit(train_term_doc,train[label_cols[0]])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
preds = clf.predict(test_term_doc)

In [35]:
preds.shape

(153164,)

In [37]:
preds[:10]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [48]:
clf.classes_

array([0, 1])

In [38]:
preds_probe = clf.predict_proba(test_term_doc)

In [57]:
preds_probe.shape

(153164, 2)

In [68]:
preds_probe[:3]

array([[ 0.00863436,  0.99136564],
       [ 0.99639687,  0.00360313],
       [ 0.98546704,  0.01453296]])

In [69]:
preds_probe[:3,1]

array([ 0.99136564,  0.00360313,  0.01453296])

In [70]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds_probe[:,1], columns = [label_cols[0]])], axis=1)
#submission.to_csv(SUB_FILE, index=False)

In [71]:
submission.head()

,id,toxic
0,00001cee341fdb12,0.991366
1,0000247867823ef7,0.003603
2,00013b17ad220c46,0.014533
3,00017563c3f7919a,0.000624
4,00017695ad8997eb,0.028256


In [22]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [136]:
m1 = np.random.randint(2, size=(4,5))
m1

array([[1, 0, 0, 0, 0],
       [1, 1, 1, 0, 1],
       [1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1]])

In [173]:
m1 = np.random.randn(4,5)
m1 = m1**2
m1

array([[  1.09353208e+00,   3.71251322e-01,   4.66296661e-01,
          4.83884151e-03,   1.20840959e-02],
       [  1.78843501e+00,   1.38341126e+00,   3.33011362e-01,
          1.16835303e-01,   4.16152321e-01],
       [  5.46027422e-02,   4.63059470e-01,   3.12053571e-01,
          3.48397312e-01,   8.63589239e-02],
       [  2.32624614e-02,   1.63624474e+00,   4.70751619e-01,
          5.55710263e-04,   1.93622833e+00]])

In [174]:
y = np.array([1,1,0,0])

In [175]:
1==y

array([ True,  True, False, False], dtype=bool)

In [176]:
m1[y==1]

array([[ 1.09353208,  0.37125132,  0.46629666,  0.00483884,  0.0120841 ],
       [ 1.78843501,  1.38341126,  0.33301136,  0.1168353 ,  0.41615232]])

In [177]:
p = m1[y==1].sum(0)
p

array([ 2.88196709,  1.75466258,  0.79930802,  0.12167414,  0.42823642])

In [178]:
(y==1).sum()

2

In [179]:
p1 = p/((y==1).sum())
p1

array([ 1.44098354,  0.87733129,  0.39965401,  0.06083707,  0.21411821])

In [180]:
m1[y==0]

array([[  5.46027422e-02,   4.63059470e-01,   3.12053571e-01,
          3.48397312e-01,   8.63589239e-02],
       [  2.32624614e-02,   1.63624474e+00,   4.70751619e-01,
          5.55710263e-04,   1.93622833e+00]])

In [181]:
p = m1[y==0].sum(0)
p

array([ 0.0778652 ,  2.09930421,  0.78280519,  0.34895302,  2.02258725])

In [182]:
p2 = p/((y==0).sum())
p2

array([ 0.0389326 ,  1.0496521 ,  0.39140259,  0.17447651,  1.01129363])

In [183]:
p1/p2

array([ 37.01225908,   0.83583055,   1.02108166,   0.34868345,   0.21172704])

In [188]:
r = np.log(p1/p2)
r

array([ 3.61124918, -0.17932938,  0.02086252, -1.05359078, -1.55245737])

In [190]:
np.multiply(m1, r)

array([[  3.94901683e+00,  -6.65762705e-02,   9.72812223e-03,
         -5.09815881e-03,  -1.87600438e-02],
       [  6.45848447e+00,  -2.48086288e-01,   6.94745536e-03,
         -1.23096598e-01,  -6.46058737e-01],
       [  1.97184108e-01,  -8.30401689e-02,   6.51022307e-03,
         -3.67068196e-01,  -1.34068548e-01],
       [  8.40065448e-02,  -2.93426759e-01,   9.82106386e-03,
         -5.85491210e-04,  -3.00591193e+00]])

In [185]:
type(x)

scipy.sparse.csr.csr_matrix

In [23]:
x = train_term_doc
test_x = test_term_doc

In [24]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [25]:
label_cols = list(train.columns[-6:])
label_cols

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [26]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [78]:
y = train.iloc[5:8].toxic.values
y

array([0, 1, 0])

In [79]:
pr(1, y)

matrix([[ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5]])

In [27]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv(SUB_FILE, index=False)

In [28]:
submission.shape

(153164, 7)

In [29]:
submission.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999973,0.049945,0.999761,0.004672,0.854442,0.256823
1,0000247867823ef7,0.001196,0.000346,0.000727,0.000084,0.001928,0.000386


import lightgbm as lgb

params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 200,
        'learning_rate': 0.1,
        'num_leaves': 127,
        'num_threads': 18, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 300,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 1.0, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'bagging_fraction': 1.0, #Randomly select part of data without resampling
        'bagging_freq': 0, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'metric' : 'auc'
    }

model_1 = lgb.train(params, train_set=lgb_train_1, valid_sets=lgb_train_1, verbose_eval=5)

lgb_train_1 = lgb.Dataset(train_term_doc, train.toxic.values)

p_1 = model_1.predict(test_term_doc)

submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols[0])], axis=1)
#submission.to_csv('submission1.csv', index=False)